### 웹툰 이미지를 다운로드 하여 로컬에 저장하기

In [16]:
import requests
import os   

req_header_dict = {
    'referer':'https://comic.naver.com/webtoon/detail?titleId=748105&no=149&weekday=thu' 
}

img_url_list = [
    'https://image-comic.pstatic.net/webtoon/748105/149/20220209180003_7730b7fb6d510e4fa7b7b1292a3ca472_IMAG01_1.jpg',
    'https://image-comic.pstatic.net/webtoon/748105/149/20220209180003_7730b7fb6d510e4fa7b7b1292a3ca472_IMAG01_2.jpg',
    'https://image-comic.pstatic.net/webtoon/748105/149/20220209180003_7730b7fb6d510e4fa7b7b1292a3ca472_IMAG01_3.jpg'
]

for img_url in img_url_list:
    res = requests.get(img_url, headers = req_header_dict)
    print(res.status_code)
    
    if res.ok:
        #binary data 가져올때 .content 속성사용
        img_data = res.content
        
        # url에서 파일명만 추출하기
        file_name = os.path.basename(img_url)
        file_name ='webtoonimg/' + file_name

        # 서버에서 가져온 binary data를 file로 저장하기
        with open(file_name, 'wb') as file:
            print(f'Write to file: {file_name} ({len(img_data)}): bytes')
            file.write(img_data)

200
Write to file: webtoonimg/20220209180003_7730b7fb6d510e4fa7b7b1292a3ca472_IMAG01_1.jpg (105043): bytes
200
Write to file: webtoonimg/20220209180003_7730b7fb6d510e4fa7b7b1292a3ca472_IMAG01_2.jpg (119375): bytes
200
Write to file: webtoonimg/20220209180003_7730b7fb6d510e4fa7b7b1292a3ca472_IMAG01_3.jpg (111900): bytes


### 웹툰의 특정회차의 모든 image 다운로드 하기
* Attribute Selector를 사용하여 jpg 파일명들을 모두 추출하여 리스트에 저장하기

In [37]:
import requests
from bs4 import BeautifulSoup

main_url = 'https://comic.naver.com/webtoon/detail?titleId=748105&no=149&weekday=thu'

res = requests.get(main_url)
print(res.status_code)

if res.ok:
    soup = BeautifulSoup(res.text, 'html.parser')
    img_tags = soup.select("img[src*='.jpg']")
    #print(img_tags)
    
    img_url_list = []
    for img_tag in img_tags:
        img_url = img_tag['src']
        
        img_url_list.append(img_url)
        
        # print(img_url_list)
        
print(len(img_url_list))
        
        

200
25


### img 디렉토리 생성하기 
### 디렉토리 생성하는 2가지 함수
* os.mkdir() / osk.makedirs()
 1. os.mkdir() 은 1개 폴더만 생성
 2. os.makedirs() 는 a/b/c 처럼 하위 폴더를 생성


In [38]:
import os

dir_path = 'img'

* 리스트를 순회하면서 image 다운로드 하기

In [42]:
for idx, img_url in enumerate(img_url_list,1):
    req_header = {'referer':main_url}
    res = requests.get(img_url, headers=req_header)
    
    #print(res.status_code)
    if res.ok:
        img_data = res.content
        file_name = os.path.basename(img_url)
        file_name = 'webtoonimg/' + file_name
        with open(file_name, 'wb') as file:
            file.write(img_data)
            

In [43]:
# 파일들이 있는 디렉토리 삭제
import shutil
import os

#img 디렉토리가 있다면
dir_path = 'img'
if os.path.exists(dir_path):
    shutil.rmtree(dir_path)

### 웹툰의 타이틀과 특정회차 url을 아규먼트 받아서 다운로드 하는 함수 구현하기

In [50]:
def download_image(title, round_url):
    import requests
    from bs4 import BeautifulSoup
    import os
    import shutil
    
    # img 폴더가 있으면 삭제하기
    dir_path = 'webtoon img'
    
    if os.path.exists(dir_path):
        shutil.rmtree(dir_path)
        
    #img 디렉토리가 없으면
    if not os.path.isdir(dir_path):
        #img\외모지상주의
        title_path = os.path.join(dir_path, title)
        print(title_path)
        os.makedirs(title_path)
    
    # img url 목록을 알아내기 위한 요청을 보내기
    res = requests.get(round_url)
    if res.ok:
        # 응답으로 받은 html텍스트를 파싱하기 위한 파서(BeautifulSoup)객체생성
        soup = BeautifulSoup(res.text, 'html.parser')
        # img태그 중에서 src 속성의 값이 .jpg로 끝나는 태그들만 선택하기
        img_tags = soup.select("img[src$='.jpg']") 
        print( len(img_tags), type(img_tags) )
        
        # 특정회차 url을 referer 헤더로 설정하기
        req_header = { 'referer':round_url }
        
        for idx, img_tag in enumerate(img_tags,1):
            #img 태그의 src 속성의 값을 추출하기
            print(f'------> 다운로드 번호 {idx}')
            img_url = img_tag['src']
            
            # jpg image 데이터 요청하기
            res_img = requests.get(img_url, headers=req_header)
            if res_img.ok:
                # image binary 데이터 가져오기
                img_data = res_img.content
                
                #img\title\thumbnail_IMAG04_be987481-7e58-478d-ae1b-c33228ccc658.jpg
                file_name = title_path + '\\' + os.path.basename(img_url)
                
                with open(file_name, 'wb') as file:
                    print(f'{file_name} ({len(img_data)}) bytes')
                    file.write(img_data)

In [51]:
download_image('독립일기','https://comic.naver.com/webtoon/detail?titleId=748105&no=148&weekday=thu')
download_image('독립일기','https://comic.naver.com/webtoon/detail?titleId=748105&no=1&weekday=thu')

webtoon img\독립일기
23 <class 'bs4.element.ResultSet'>
------> 다운로드 번호 1
webtoon img\독립일기\thumbnail_IMAG04_2b479f04-a16b-4fa2-9a05-bc60cd84022c.jpg (15357) bytes
------> 다운로드 번호 2
webtoon img\독립일기\20220203154235_037a3e60ac006660a88f7e3b5b9bb37f_IMAG01_1.jpg (150355) bytes
------> 다운로드 번호 3
webtoon img\독립일기\20220203154235_037a3e60ac006660a88f7e3b5b9bb37f_IMAG01_2.jpg (124091) bytes
------> 다운로드 번호 4
webtoon img\독립일기\20220203154235_037a3e60ac006660a88f7e3b5b9bb37f_IMAG01_3.jpg (136037) bytes
------> 다운로드 번호 5
webtoon img\독립일기\20220203154235_037a3e60ac006660a88f7e3b5b9bb37f_IMAG01_4.jpg (115287) bytes
------> 다운로드 번호 6
webtoon img\독립일기\20220203154235_037a3e60ac006660a88f7e3b5b9bb37f_IMAG01_5.jpg (149182) bytes
------> 다운로드 번호 7
webtoon img\독립일기\20220203154235_037a3e60ac006660a88f7e3b5b9bb37f_IMAG01_6.jpg (164473) bytes
------> 다운로드 번호 8
webtoon img\독립일기\20220203154235_037a3e60ac006660a88f7e3b5b9bb37f_IMAG01_7.jpg (101341) bytes
------> 다운로드 번호 9
webtoon img\독립일기\20220203154235_037a3e60ac0066